In [ ]:
# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

import time
import matplotlib.pyplot as plt

In [ ]:
from data_helpers.wine_quality_data_helper import load_wine_quality_data
from data_helpers.mnist_data_helper import load_mnist_data

In [ ]:
class GaussianMixtureWrapper:
    NAME = 'GMM'
    def __init__(self, n_components):
        self.n_components = n_components
        self.model = GaussianMixture(n_components=n_components)

    def fit(self, X):
        self.model.fit(X)

    def predict(self, X):
        return self.model.predict(X)
    
    def bic(self, X):
        return self.model.bic(X)

    def aic(self, X):
        return self.model.aic(X)

class KMeansWrapper:
    NAME = 'KMeans'
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.model = KMeans(n_clusters=n_clusters)

    def fit(self, X):
        self.model.fit(X)

    def predict(self, X):
        return self.model.predict(X)
    
    def inertia(self):
        return self.model.inertia_

In [ ]:
# DATASET = 'mnist'
DATASET_NAME = 'wine_quality'
dataset_to_str = {'mnist': 'MNIST', 'wine_quality': 'Wine Quality'}
DATASET_STR = dataset_to_str[DATASET_NAME]

# DIM_RED_METHODS = ['ICA', 'PCA']
# DIM_RED_METHOD = 'ICA'
DIM_RED_METHOD = 'PCA'

METHOD = GaussianMixtureWrapper
# METHOD = KMeansWrapper
SAVE = True

if DATASET_NAME == 'mnist':
    num_components = None # ????
elif DATASET_NAME == 'wine_quality':
    num_components = 5

In [ ]:
num_components

In [ ]:
import pickle
if DATASET_NAME == 'wine_quality':
    DATASET_STR = 'Wine Quality'
    X_train = pickle.load(open(f'transformed_data/{DATASET_NAME}/{DIM_RED_METHOD}/X_train.pkl', 'rb'))
    y_train = pickle.load(open(f'transformed_data/{DATASET_NAME}/{DIM_RED_METHOD}/y_train.pkl', 'rb'))
    X_test = pickle.load(open(f'transformed_data/{DATASET_NAME}/{DIM_RED_METHOD}/X_test.pkl', 'rb'))
    y_test = pickle.load(open(f'transformed_data/{DATASET_NAME}/{DIM_RED_METHOD}/y_test.pkl', 'rb'))
elif DATASET_NAME == 'mnist':
    DATASET_STR = 'MNIST'
    X_train = pickle.load(open(f'transformed_data/{DATASET_NAME}/{DIM_RED_METHOD}/X_train.pkl', 'rb'))
    y_train = pickle.load(open(f'transformed_data/{DATASET_NAME}/{DIM_RED_METHOD}/y_train.pkl', 'rb'))
    X_test = pickle.load(open(f'transformed_data/{DATASET_NAME}/{DIM_RED_METHOD}/X_test.pkl', 'rb'))
    y_test = pickle.load(open(f'transformed_data/{DATASET_NAME}/{DIM_RED_METHOD}/y_test.pkl', 'rb'))
else:
    raise ValueError(f'Invalid dataset name {DATASET_NAME}')

In [ ]:
model = METHOD(num_components)
model.fit(X_train)
pred = model.predict(X_test)

In [ ]:
buckets = {}
for i in range(num_components):
    mask = pred == i
    y_masked = y_test[mask]
    buckets[i] = y_masked

In [ ]:
# sort buckets by size (largest to smallest)
buckets = {k: v for k, v in sorted(buckets.items(), key=lambda item: len(item[1]), reverse=True)}

In [ ]:
YELLOW = np.array([1, 1, 0])
RED = np.array([1, 0, 0])
LABELS = [3, 4, 5, 6, 7, 8, 9]
COLORS = {}
for i, label in enumerate(LABELS):
    COLORS[label] = YELLOW + i * (RED - YELLOW) / len(LABELS)

In [ ]:
plt.figure(figsize=(10, 6))
for bucket_num, labels in buckets.items():
    bottom = 0
    for label_num in sorted(np.unique(labels)):
        l = sum(labels == label_num)
        plt.bar(bucket_num, l, bottom=bottom, color=COLORS[label_num])
        bottom += l

# get variance
weighted_var = 0
for bucket_num, labels in buckets.items():
    if len(labels) == 0:
        continue
    weighted_var += np.var(labels) * len(labels)
weighted_var /= len(y_test)

fontsize = 14
plt.title(f'{DATASET_STR} - DimRed={DIM_RED_METHOD} - GMM Components (Variance = {weighted_var:.4f})', fontsize=fontsize)
plt.xlabel('Cluster', fontsize=fontsize)
plt.ylabel('Count', fontsize=fontsize)

plt.xlim(-0.5, 4.5)

custom_lines = [plt.Line2D([0], [0], color=COLORS[i], lw=5) for i in LABELS]
plt.legend(custom_lines, LABELS, fontsize=fontsize)